In [12]:
import datetime
import os

os.environ["PATH"] = (
    "/opt/apps/rhel9/cuda-12.4/bin:/opt/apps/rhel9/cuda-12.4/" + os.environ["PATH"]
)
os.environ["LD_LIBRARY_PATH"] = (
    "/opt/apps/rhel9/cuda-12.4/bin:/opt/apps/rhel9/cuda-12.4"
    + os.environ.get("LD_LIBRARY_PATH", "")
)

from huggingface_hub import HfApi

import numpy as np
import numpy.random as npr
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch import nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import (  # AdamW,
    BertForSequenceClassification,
    BertModel,
    BertTokenizer,
    get_linear_schedule_with_warmup,
)

pd.set_option("mode.copy_on_write", True)
repo_id = "nickeubank/leaa_grant_subjects"
workingdir = "/hpc/group/ssri/nce8/leaa_subj/"

/hpc/group/ssri/nce8/miniforge3/envs/torch/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
assert torch.cuda.is_available()

In [14]:
# dir = "https://github.com/nickeubank/leaa_subj/raw/refs/heads/main/"
grants = pd.read_parquet(workingdir + "subj_text_and_labels.parquet")

#########
# Split into train test and for predict
#########
grants = grants.drop_duplicates("description")

labeled = grants[grants["label_1"].notnull()]
labeled["label_1_encoded"] = labeled["label_1"] - 1

In [15]:
labeled = labeled.sort_values("description")

train_label, test_label, train_text, test_text = train_test_split(
    labeled["label_1_encoded"].values,
    labeled["description"].values,
    test_size=0.2,
    random_state=45,
    stratify=labeled["label_1"],
)
print(len(train_label))
print(len(test_label))

64148
16038


In [16]:
########
# Preprocess
########


class ClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long),
        }

In [17]:
# Hypertuning Parameters
# hypers = {"lr": [], "mlen": [], "batch_size": [], "accuracy": []}
# hypers = pd.read_parquet("hyperparams_1digit_bigbert.parquet").to_dict()
# for k in hypers.keys():
#     hypers[k] = list(hypers[k].values())

# df = pd.read_parquet("hyperparams_1digit_bigbert.parquet")
# df.sort_values("accuracy", ascending=False)

# params = [
#     {"mlen": mlen, "batch_size": batch_size, "lr": lr}
#     for batch_size in [16, 32]
#     for mlen in [128, 256, 512]
#     for lr in [1e-7, 1e-6, 1e-5, 1e-4]
# ]
# params

In [17]:
# for p in params[11:]:
# MAX_LEN = p["mlen"]
# BATCH_SIZE = p["batch_size"]
# EPOCHS = 5
# LEARNING_RATE = p["lr"]

MAX_LEN = 256
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 0.000010

model = "bert-large-uncased"
tokenizer = BertTokenizer.from_pretrained(model)
train_dataset = ClassificationDataset(train_text, train_label, tokenizer, MAX_LEN)
test_dataset = ClassificationDataset(test_text, test_label, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Model and Device Setup
assert torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BertForSequenceClassification.from_pretrained(
    model, num_labels=grants["label_1"].nunique()
)
model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    loop = tqdm(train_loader, leave=True)

    if (epoch > 0) and (epoch % 3 == 0):

        time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")

        model.push_to_hub(
            repo_id, commit_message=f"checkpoint_large_{time}_epoch{epoch}"
        )
        tokenizer.push_to_hub(
            repo_id,
            commit_message=f"checkpoint_large_{time}_epoch{epoch}",
        )

    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
        )
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch} Loss: {total_loss / len(train_loader)}")

time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")
model.push_to_hub(repo_id, commit_message=f"trained_largebert_{time}_epoch{epoch}")
tokenizer.push_to_hub(
    repo_id,
    commit_message=f"trained_largebert_{time}_epoch{epoch}",
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 0: 100%|██████████| 1003/1003 [11:49<00:00,  1.41it/s, loss=1.39]


Epoch 0 Loss: 1.4512595287408097


Epoch 1: 100%|██████████| 1003/1003 [11:49<00:00,  1.41it/s, loss=1.53]


Epoch 1 Loss: 1.4740087474925687


Epoch 2: 100%|██████████| 1003/1003 [11:49<00:00,  1.41it/s, loss=1.28]


...

Epoch 3 Loss: 0.20883439047506963


Epoch 4: 100%|██████████| 502/502 [11:15<00:00,  1.35s/it, loss=0.0218] 


Epoch 4 Loss: 0.14877604184100826
Validation Accuracy: 0.8749
{'lr': [1e-07, 1e-06, 1e-05, 0.0001, 1e-07, 1e-06, 1e-05, 0.0001, 1e-07, 1e-07, 1e-06, 1e-05, 0.0001, 1e-07, 1e-06, 1e-05, 0.0001, 1e-07, 1e-06, 1e-07, 1e-07, 1e-06, 1e-05, 0.0001, 1e-07, 1e-06, 1e-05, 0.0001, 1e-07, 1e-06, 1e-05, 0.0001, 1e-07, 1e-06, 1e-05, 0.0001, 1e-07, 1e-06, 1e-05], 'mlen': [128, 128, 128, 128, 128, 128, 128, 128, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 512, 512, 512, 512, 512, 512, 512, 512, 512, 128, 128, 128, 128, 256, 256, 256, 256, 512, 512, 512], 'batch_size': [8, 8, 8, 8, 16, 16, 16, 16, 8, 8, 8, 8, 8, 16, 16, 16, 16, 32, 32, 8, 8, 8, 8, 8, 16, 16, 16, 16, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32], 'accuracy': [0.7602419405125647, 0.8706740662218619, 0.8705493546174472, 0.4105506017334913, 0.6138928727318077, 0.8669327180894182, 0.8797780133441416, 0.4105506017334913, 0.7128515308349442, 0.7245744216499346, 0.8767225790359793, 0.875413107189624, 0.4079940138429881, 0.67175905718

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/502 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB. GPU 0 has a total capacity of 31.50 GiB of which 172.12 MiB is free. Including non-PyTorch memory, this process has 31.33 GiB memory in use. Of the allocated memory 28.32 GiB is allocated by PyTorch, and 2.64 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)

        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Validation Accuracy: {accuracy:.4f}")

# hypers["lr"].append(LEARNING_RATE)
# hypers["mlen"].append(MAX_LEN)
# hypers["batch_size"].append(BATCH_SIZE)
# hypers["accuracy"].append(accuracy)
# print(hypers)
# pd.DataFrame(hypers).to_parquet(workingdir + "hyperparams_1digit_bigbert.parquet")

In [17]:
# # Look at fine tuning
# df = pd.DataFrame(hypers).sort_values("accuracy", ascending=False)
# df.groupby("lr")["accuracy"].mean()